In [ ]:
import os
from textwrap import wrap
from random import sample, randint
import numpy as np
import shutil
from Bio import SeqIO
bases = ['A','C','G','T']
lengths = [1500+(np.random.poisson(lam=10)*randint(1, 1000)) for i in range(0,1000)]
binners = ['binner_1', 'binner_2', 'binner_3', 'binner_4']
shutil.rmtree("test_data/simulation", ignore_errors=True)
os.makedirs("test_data/simulation/bins", exist_ok=True)

In [ ]:
with open("test_data/simulation/assembly.fasta", "w") as fout:
    for i, l in enumerate(lengths):
        seq = np.random.choice(bases, size=l, replace=True)
        seq = "".join(seq)
        fout.write(f">seq_{i}\n")
        #for line in wrap(seq, 68):
        fout.write(f"{seq}\n")

In [ ]:
for binner in binners:
    os.makedirs(f"test_data/simulation/bins/{binner}/", exist_ok=True)
    nbins = randint(10, 100)
    ios = {i: open(f"test_data/simulation/bins/{binner}/bin_{i}.fa", "w") for i in range(0, nbins)}
    for i, record in enumerate(SeqIO.parse("test_data/simulation/assembly.fasta", "fasta")):
        if i < nbins:
            SeqIO.write(record, ios[i], "fasta")
        else:
            SeqIO.write(record, ios[sample(ios.keys(), 1)[0]], "fasta") 

In [ ]:
import hashlib
from glob import glob
def md5(fname):
    hash_md5 = hashlib.md5()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

# check checksums between simulated and restored
if os.path.exists("test_data/simulation/restore"):
    fls = glob("test_data/simulation/restore/binner*/*.fa")
    for f in fls:
        if md5(f.replace("/restore/","/bins/")) != md5(f):
            print("missmatch", f, md5(f.replace("/restore/","/bins/")), md5(f))
        else:
            print("Same", f, md5(f.replace("/restore/","/bins/")), md5(f))